In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

R[write to console]: 
Attaching package: ‘Seurat’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: Signac built for for SeuratObject v4 was just loaded with SeuratObject
v5; disabling v5 assays and validation routines, and ensuring assays
work in strict v3/v4 compatibility mode



ModuleNotFoundError: No module named 'tools.utils.utils'

In [5]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
# input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# input = "/ps/ai-ready/data/error/tung.rds"
adata = load_anndata(input)
adata

/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad


AnnData object with n_obs × n_vars = 2392 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'scale.data'

In [6]:
adata.obs['seurat_clusters'].unique()

['3', '5', '4', '0', '8', '2', '6', '1', '7', '9']
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [13]:
coords

,0,1
10X_P4_3_AAAGTAGAGATGCCAG,8.023769,-3.150253
10X_P4_3_AACTCCCGTCGGGTCT,9.334201,4.313109
10X_P4_3_AACTCTTAGTTGCAGG,-10.855099,1.073269
10X_P4_3_AACTCTTTCATAACCG,10.493811,8.358018
10X_P4_3_AAGACCTAGATCCGAG,-0.852234,-14.554533
...,...,...
10X_P7_7_TTGTAGGTCATGTAGC,-5.544025,-0.052144
10X_P7_7_TTTACTGGTGTTGAGG,6.659881,2.071705
10X_P7_7_TTTCCTCCATACCATG,7.218575,3.513279
10X_P7_7_TTTGTCAGTTACCGAT,-9.230332,1.550699


In [18]:
for i, val in enumerate(sorted(adata.obs['seurat_clusters'].unique())):
    a = adata.obs[adata.obs['seurat_clusters'] == val]
    b = coords[adata.obs['seurat_clusters'] == val]
    print(a['cell_ontology_class'].astype(str))

10X_P4_3_AACTCTTTCATAACCG    bladder cell
10X_P4_3_AGTGTCACAGTATCTG    bladder cell
10X_P4_3_CATTCGCTCTCGTATT    bladder cell
10X_P4_3_CCTTCGATCTCCGGTT    bladder cell
10X_P4_3_CGGGTCATCGTGGACC    bladder cell
                                 ...     
10X_P4_4_TTTGCGCTCACCTCGT    bladder cell
10X_P4_4_TTTGGTTTCGGTCCGA    bladder cell
10X_P4_4_TTTGTCACAGACACTT    bladder cell
10X_P7_7_GGCGTGTAGTCCTCCT    bladder cell
10X_P7_7_GTCGGGTAGCGATCCC    bladder cell
Name: cell_ontology_class, Length: 464, dtype: object
10X_P4_3_CACACCTGTTTGACAC    bladder urothelial cell
10X_P4_3_CTGCCTACAGCCACCA    bladder urothelial cell
10X_P4_3_CTTGGCTCACGAAACG    bladder urothelial cell
10X_P4_3_GCATGATCATATGGTC    bladder urothelial cell
10X_P4_3_GCTGCTTTCAAAGTAG    bladder urothelial cell
                                      ...           
10X_P7_7_CACAGGCTCAGGCAAG    bladder urothelial cell
10X_P7_7_CACCACTGTCATGCAT    bladder urothelial cell
10X_P7_7_CATATGGTCGGCATCG    bladder urothelial cell
10X_P7_

In [8]:
coords = pd.DataFrame(adata.obsm['X_umap'], index=adata.obs.index)
coords

,0,1
10X_P4_3_AAAGTAGAGATGCCAG,8.023769,-3.150253
10X_P4_3_AACTCCCGTCGGGTCT,9.334201,4.313109
10X_P4_3_AACTCTTAGTTGCAGG,-10.855099,1.073269
10X_P4_3_AACTCTTTCATAACCG,10.493811,8.358018
10X_P4_3_AAGACCTAGATCCGAG,-0.852234,-14.554533
...,...,...
10X_P7_7_TTGTAGGTCATGTAGC,-5.544025,-0.052144
10X_P7_7_TTTACTGGTGTTGAGG,6.659881,2.071705
10X_P7_7_TTTCCTCCATACCATG,7.218575,3.513279
10X_P7_7_TTTGTCAGTTACCGAT,-9.230332,1.550699


In [3]:
info=adata.__str__()

In [4]:
print(info)

AnnData object with n_obs × n_vars = 9424 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'organ_tissue_colors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'


In [42]:
adata.obs.free_annotation.unique()

['macrophage', 'stromal cells', 'CD8-positive memory T cell', 'classic dendritic cells type 1', 'alpha-beta T cell', ..., 'Memory B cell', 'Langerhans cells', 'Naive B cell', 'smooth muscle cells', 'skeletal muscle cells']
Length: 25
Categories (25, object): ['CD4-positive helper T cells', 'CD4-positive memory T cell', 'CD8-positive cytotoxic T cells', 'CD8-positive memory T cell', ..., 'muscle cells', 'skeletal muscle cells', 'smooth muscle cells', 'stromal cells']

In [44]:
def subset_by_obskey(
        adata: AnnData,
        obskey: str,
        values: list):
    adata_merge = None
    if isinstance(adata, AnnData):
        if obskey in adata.obs.keys() and len(values)>0:
            adata_list = []
            for value in values:
                adata_list.append(adata[adata.obs[obskey] == value])
            adata_merge = sc.AnnData.concatenate(*adata_list, join='outer')
            adata_merge.var_names_make_unique()
            adata_merge.obs_names_make_unique()

    return adata_merge

In [45]:
adata_sub = subset_by_obskey(adata, "free_annotation", ['macrophage', 'stromal cells', 'CD8-positive memory T cell', 'classic dendritic cells type 1'])
adata_sub

AnnData object with n_obs × n_vars = 4663 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'batch'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'

In [30]:
# adata_sub = adata[adata.obs.free_annotation in ["stromal cell", "macrophage", "CD8-positive memory T cell", "classic dendritic cells type 1	"], :].cpoy()
# adata_sub = adata[adata.obs['free_annotation']=="stromal cell" | adata.obs['free_annotation']=="macrophage", :].copy()
adata_sub = adata[adata.obs['free_annotation']=="macrophage", :].copy()
adata_sub

AnnData object with n_obs × n_vars = 846 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'organ_tissue_colors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [20]:
import hashlib

def get_file_md5(path: str, split_num=25600, get_byte=8):

    if not isinstance(split_num, int) or split_num <= 0:
        raise TypeError("split_num must be a positive none-zero integer!")
    if not isinstance(get_byte, int) or get_byte <= 0:
        raise TypeError("get_byte must be a positive none-zero integer!")
    if not os.path.exists(path):
        raise TypeError("%s does not exist!" % path)
    if os.path.isdir(path):
        raise TypeError("%s is a folder, while path should be a file!" % path)
    
    size = os.path.getsize(path)
    # For a small file (equal to or less than 500M), caculate the MD5 values directly.
    if size < split_num * get_byte:
        # Read the file
        with open(path, 'rb') as f1:
            f1 = f1.read()
        cipher = hashlib.md5()
        cipher.update(str(split_num).encode('utf-8'))
        cipher.update(f1)
        cipher.update(str(get_byte).encode('utf-8'))
        return cipher.hexdigest()
    # For a large file, split the file in to several segment, and then sum the MD5 value. 
    mean_size = size // split_num
    cipher = hashlib.md5()
    # Position
    place = 0
    with open(path, 'rb') as f1:
        for i in range(split_num):
            f1.seek(place)
            res = f1.read(get_byte)
            cipher.update(res)
            place = place + mean_size

    return cipher.hexdigest()


def get_md5(path:str):
    md5 = []
    if not os.path.exists(path):
        raise TypeError("%s does not exist!" % path)
    if os.path.isdir(path):
        for file in os.listdir(path):
            md5.append(get_file_md5(path+file))
    else:
        md5.append(get_file_md5(path))
    return md5

In [12]:
md5 = get_md5("/ps/ai-ready/data/Clustering/Tabula_Sapiens/")
md5

['442217c862bbe96ca6f524773688197d',
 'bff56ec0a9c9214d5a25376c3c3e4769',
 'a24e191e516f736c86490395a3d2476f',
 '765a3f8472b32a8e48e5cd17d29e6f2a',
 '7d7063ee0b6b865e6f4147e3cde21a1c',
 '9529fb778f3be8160ee0535b1ad05d8c',
 '83ced3bcf724cd6a69295661fbba570a',
 'a3b6a2e86e360676f069876105f81caa',
 '836672abf651ccac3dc1a731a4a6a6e2',
 'd89dd623864922970e989223fbf747ef',
 '9e80b231464c8374f96f7b7028421940',
 '14a66c3744a70d220410fc2a67bcbe16',
 '405d16ce0dcbbdd6f81bb38c2e4fdf5a',
 'd6017480f597f082e5d36eaf0c71a076',
 'd5a166995b294ba78a654b96a43ec2a4',
 'ac9f2d95d876bbc23d2ede3580b46f1e',
 'a176808a1074d052fb184f769fa295ad',
 'd71bfdff4c63d7504c9c054ffd8ef896',
 'ffad55079d0a0b5a0e9a74492d78a5b1',
 'a57f84ceabb70fa59e31a8cdfbf9d249',
 '158b140b43ffeae4c8087169c714a6a9',
 '0ef573b467c9e98816634534ea4e1e7c',
 '29d84f4c9bea2cdf36722236bc3b91c7',
 'ea4eb0b890f2a1234e36353136d22223',
 '3d0eff1ead778b9f2ec2f27fb81be51a',
 '68ae8de8a14afaaa79a49e8d1c9987d3',
 'f43db9de4991761a713e77909f9fb539',
 

In [8]:
os.listdir("/ps/ai-ready/data/Clustering/Tabula_Sapiens/")

['TS_Small_Intestine.h5ad',
 'TS_Skin_test.h5ad',
 'TS_Heart.h5ad',
 'Tabula_Sapiens_metadata.csv',
 'TS_endothelial.h5ad',
 'TS_Salivary_Gland.h5ad',
 'TS_Skin.h5ad',
 'TS_Tongue.h5ad',
 'TS_Kidney.h5ad',
 'TS_Bone_Marrow.h5ad',
 'TS_Trachea.h5ad',
 'TS_Muscle.h5ad',
 'TS_Pancreas.h5ad',
 'TS_epithelial.h5ad',
 'TS_Eye.h5ad',
 'TS_Lymph_Node.h5ad',
 'TabulaSapiens.h5ad',
 'TS_stromal.h5ad',
 'TS_Uterus.h5ad',
 'TS_germ line.h5ad',
 'TS_Large_Intestine.h5ad',
 'TS_Vasculature.h5ad',
 'TS_Lung.h5ad',
 'TS_Spleen.h5ad',
 'TS_Mammary.h5ad',
 'TS_Bladder.h5ad',
 'TS_Thymus.h5ad',
 'TS_Fat.h5ad',
 'TS_immune.h5ad',
 'TS_Skin_test.h5seurat',
 'TS_Prostate.h5ad',
 'TS_Liver.h5ad',
 'TS_Blood.h5ad']

In [ ]:
adata.obs.head()

In [ ]:
adata.var.head()

In [ ]:
df = pd.DataFrame.sparse.from_spmatrix(adata.X, index=None, columns=None)
df

In [ ]:
# Convert Seurat/Single-Cell Experiment object to Anndata object and return the path of Anndata object
def convert_seurat_sce_to_anndata1(path, assay='RNA'):
    import rpy2.rinterface_lib.callbacks as rcb
    import rpy2.robjects as ro
    import anndata2ri
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter
    
    rcb.logger.setLevel(logging.ERROR)
    ro.pandas2ri.activate()
    anndata2ri.activate()

    # Defining the R script and loading the instance in Python
    ro.r['source'](os.path.abspath("../tools/formating/formating.R"))
    # Access the loaded R functions
    ConvertSeuratSCEtoAnndata_r = ro.globalenv['ConvertSeuratSCEtoAnndata']

    assay_names = None
    adata_path = None
    results = None

    if path.endswith(".h5Seurat") or path.endswith(".h5seurat") or path.endswith(".rds") or path.endswith(".Robj"):
        results = ConvertSeuratSCEtoAnndata_r(path, assay=assay)
        print(results)
        adata_path = str(results.rx2('anndata_path'))
        # adata_path = list(results[2])[0]
        assay_names = list(results[1])
    
    return results

In [ ]:
results = convert_seurat_sce_to_anndata1(input)
results

In [ ]:
type(results)

In [ ]:
from collections import OrderedDict

results = OrderedDict(results)

In [ ]:
results['anndata_path'][0]